<font size =8> Barnacle Detection by Training a Model
<font size =4>

In this experiment, I will tune a large pre-trained model. YOLO11 model is known for having exceptional object detection and counting abilities. By training it on a dataset of just Barnacles, I optimize its capabilities for the specific task of counting barnacles. I obtained the datatset of Barnacles from [Roboflow Universe](https://universe.roboflow.com/), world's largest collection of open source computer vision datasets and APIs.

In [4]:
from ultralytics import YOLO
from helpers import *
import os 

<font size =5>Training and initializing the model 

<font size =4> The YOLO model was trained using a Barnacle Detection dataset for 40 epochs on Google Collab. The best weight  were downloaded and stored in the best.pt file in this directory. The YOLO model uses this set of weights for the inference section. If interested on the specifics of the training, you can check out the YOLO11.ipynb notebook, or the Training_Results directory.


In [5]:
model = YOLO('best.pt')
results = model('sample_image.png')

object_count = len(results[0].boxes)
print(f" The number of barnacles is: {object_count}")


image 1/1 c:\Users\arses\Barnacles\scripts\sample_image.png: 640x640 300 barnacles, 227.2ms
Speed: 9.4ms preprocess, 227.2ms inference, 16.6ms postprocess per image at shape (1, 3, 640, 640)
 The number of barnacles is: 300


<font size = 4> While performing inference on the entire image containing 1000+ barnacles, the model grossly underperforms. This is because the dataset on which the model was trained comprises of images with relatively much lower number of barnacles. 

To tackle this issue, I take a similar approach as with the API-model, breaking the image down into ~30 images. The images are then stored in a temporary directory in the notebook and deleted upon the completion of the inference task. 

<font size = 5>Specifying Image Path and Output Directory

In [6]:
output_directory  = "C:/Users/arses/Desktop/Cropped_images"
image_path = "sample_image.png"

try : 
    os.mkdir(output_directory)
    print(f"Directory '{output_directory}' created successfully.")

#Already created this file
except FileExistsError: 
    print(f"Directory '{output_directory}' already exists.")

#Check access and permissions settings 
except PermissionError: 
    print(f"Permission denied: Unable to create '{output_directory}.")

#Any other type of errors 
except Exception as e: 
    print(f"An error occured:{e}")

Directory 'C:/Users/arses/Desktop/Cropped_images' already exists.


<font size =5>Crop the image into ~30 smaller tiles

In [7]:
crop_image_into_tiles(image_path,output_directory)

Image cropped successfully


<font size = 5>Running the loop for the counting the number of barnacles

In [8]:
number_of_barnacles= 0
number_of_images= directory_size(output_directory)
for i in range(number_of_images):
    
    result = model(f"{output_directory}/tile_{i}.png", verbose= False)
    count = len(result[0].boxes)
    number_of_barnacles+= count

print(f"The number of barnacles present in the image is: {number_of_barnacles}")


Number of files in 'C:/Users/arses/Desktop/Cropped_images': 30
The number of barnacles present in the image is: 1221


<font size = 5>Removing the saved tiles from the directory 

In [9]:
remove_files_from_directory(output_directory)